We will create a deep neural network with Keras.

In [1]:
# import some modules we may need

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# try out PCA as well as some feature selection. or try both!
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Activation

/Users/rajsingh/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read in the csv file and put it in a DataFrame.
df_train = pd.read_csv('audio_training_data_cleaned.csv')

In [3]:
# drop any null values we may have forgotten
df_train = df_train.dropna(how='any',axis=0)

In [8]:
# split into X_train and y_train
X_train = df_train.drop(columns=['filename','age','Unnamed: 0']).values
y_train = df_train['age']

# to do feature selection, must convert categorical values to
# numerical values
replaced = {'teens':1,'twenties':2,'thirties':3,'fourties':4,'fifties':5,'sixties':6,
            'seventies':7,'eighties':8}
y_train_encoded = y_train.replace(replaced)

In [9]:
# use LASSO technique to do some feature selection because there are 170 features
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, y_train_encoded)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X_train)
print(X_new.shape) # our new shape has decreased to 81 features

(73765, 81)


In [27]:
# try normal 3-layer perceptron
from keras.utils import to_categorical
y_train_keras_encoded = to_categorical(y_train_encoded)[:,1:] # adds unneeded extra column
DL_model = Sequential()

DL_model.add(Dense(200, activation='relu', input_shape = (X_new.shape[1],)))

DL_model.add(Dense(200, activation='relu'))

DL_model.add(Dense(8, activation='softmax'))

DL_model.compile(optimizer='adam',loss='categorical_crossentropy',
             metrics=['accuracy'])
DL_model.fit(X_new,y_train_keras_encoded,epochs=50,batch_size=100)

Epoch 1/50
73765/73765 [==============================] - 2s 33us/step - loss: 1.7162 - acc: 0.3187
Epoch 2/50
73765/73765 [==============================] - 2s 28us/step - loss: 1.6092 - acc: 0.3661
Epoch 3/50
73765/73765 [==============================] - 2s 28us/step - loss: 1.5581 - acc: 0.3869
Epoch 4/50
73765/73765 [==============================] - 2s 29us/step - loss: 1.5008 - acc: 0.4124
Epoch 5/50
73765/73765 [==============================] - 2s 30us/step - loss: 1.4471 - acc: 0.4353
Epoch 6/50
73765/73765 [==============================] - 2s 31us/step - loss: 1.3882 - acc: 0.4626
Epoch 7/50
73765/73765 [==============================] - 2s 30us/step - loss: 1.3305 - acc: 0.4879
Epoch 8/50
73765/73765 [==============================] - 2s 29us/step - loss: 1.2757 - acc: 0.5157
Epoch 9/50
73765/73765 [==============================] - 2s 30us/step - loss: 1.2248 - acc: 0.5365
Epoch 10/50
73765/73765 [==============================] - 2s 30us/step - loss: 1.1787 - acc: 0.5568